In [1]:
import logging
from filecmp import dircmp
from os import listdir
import os
import filecmp
from dotenv import load_dotenv
from intake import open_catalog
import matplotlib as plt
import pandas as pd
import numpy as np
from shapely import wkt
import shapely
from sqlalchemy import create_engine
from shapely.geometry import shape
from shapely.ops import unary_union
from geopandas import GeoDataFrame
import geopandas as gpd
from datetime import datetime
import networkx as nx
from shapely.geometry import MultiPolygon
import datetime as dt

load_dotenv()

True

In [2]:
catfeux = open_catalog(f'{os.getenv("PROJECT_PATH")}Fire_Detection_Data_Quality.yaml')

surfdetect_control = catfeux.sentinel_surfaces_detectees_brute.read()

## Find detection type : "Mono detection" or "pluri detection"

In [30]:
def find_intersecting_id(row, gdf):

    possible_matches_index = list(gdf.sindex.intersection(row['geometry'].bounds))
    possible_matches = gdf.iloc[possible_matches_index]
    precise_matches = possible_matches[possible_matches.geometry.intersects(row['geometry'])]
    intersecting_ids = precise_matches['surface_id'].tolist()
    intersecting_ids = [id_ for id_ in intersecting_ids if id_ != row['surface_id']]
    return intersecting_ids

In [38]:
def mesure_pluri_detection(df):
    pluri_detection_list=df[df['groupe_id'].notna()]
    pluri_detection_surface=pluri_detection_list.dissolve() ## number of mono detected detected area (ha)
    pluri_detection_surface=pd.Series(pluri_detection_surface.area/10000)
    pluri_detection_surface=pluri_detection_surface.reset_index(drop=True)
    pluri_detection_surface=pluri_detection_surface[0]

    pluri_detection_group=pluri_detection_list['groupe_id'].nunique() ## number of group
    pluri_tile_number = pd.DataFrame(pluri_detection_list["nom"].value_counts())
    
    pluri_tile_surface=pluri_detection_list.dissolve(by='nom')
    pluri_tile_surface=pd.DataFrame(pluri_tile_surface.area/10000)
    pluri_tile_surface['nom']=pluri_tile_surface.index
    pluri_tile_surface=pluri_tile_surface.reset_index(drop=True)
    
    return(pluri_tile_surface,pluri_tile_number,pluri_detection_group,pluri_detection_surface)

def mesure_mono_detection(df):
    mono_detection_list=df[df['groupe_id'].isna()]
    mono_detection_surface=mono_detection_list['surface'].sum() ## number of mono detected detected area (ha)
    mono_detection_group=mono_detection_list['groupe_id'].isna().sum() ## number of mono detected polygons

    mono_tile_number = pd.DataFrame(mono_detection_list["nom"].value_counts()) ## number of detection per tiles
    mono_tile_surface = mono_detection_list.groupby('nom')['surface'].sum().reset_index() ## sum of burned area detected per tile
    
    return(mono_tile_surface,mono_tile_number,mono_detection_group,mono_detection_surface)

In [39]:
import panel as pn
import numpy as np
from bokeh.models import HoverTool
from bokeh.palettes import RdYlBu11 as palette
from bokeh.models import LogColorMapper
from bokeh.tile_providers import OSM, get_provider
from bokeh.plotting import figure
import datetime as dt
from odc.stac import configure_rio, stac_load
import matplotlib.pyplot as plt
import holoviews as hv

pn.extension()
pn.extension('tabulator')

stylesheet = """
.tabulator-cell {
    font-size: 20px;
}
"""

custom_style = {
    'background': '#f9f9f9',
    'border': '1px solid black',
    'padding': '10px',
    'box-shadow': '5px 5px 5px #bcbcbc'
}
    
def highlight_max(s):
    '''
    highlight the maximum in a Series yellow.
    '''
    is_max = s == s.max()
    return ['background-color: green' if v else '' for v in is_max]


### PAGE 1 #########
############ table

def maj_table(date_range):
    hv.extension('bokeh')

    df=surfdetect_control.loc[(surfdetect_control['date_'] >= date_range[0]) & (surfdetect_control['date_'] <= date_range[1])]
    df['groupe_id'] = np.nan

    G = nx.Graph()

    for index, row in df.iterrows():
        intersecting_ids = find_intersecting_id(row, df)
        for id_ in intersecting_ids:
            G.add_edge(row['surface_id'], id_)

    groupes = list(nx.connected_components(G))

    for groupe_id, groupe in enumerate(groupes):
        for id_ in groupe:
            df.loc[df['surface_id'] == id_, 'groupe_id'] = groupe_id

    pluri_tile_surface,pluri_tile_number,pluri_detection_group,pluri_detection_surface=mesure_pluri_detection(df)
    mono_tile_surface,mono_tile_number,mono_detection_group,mono_detection_surface=mesure_mono_detection(df)

    info_surfaces = pd.merge(mono_tile_number, mono_tile_surface, on='nom', how='outer')
    info_surfaces = pd.merge(info_surfaces, pluri_tile_number, on='nom', how='outer')
    info_surfaces = pd.merge(info_surfaces, pluri_tile_surface, on='nom', how='outer')

    info_surfaces=info_surfaces.rename(columns={'nom':'Tile name','count_x':'Number of mono detection','surface':'Sum of mono detected area','count_y':'Number of pluri detection',0:'Sum of pluri detected area'})

    table = pn.widgets.Tabulator(info_surfaces, name="Informations à l'échelle des tuiles Sentinel-2",header_align='center', show_index=False,
                stylesheets=[stylesheet])
    table.style.apply(highlight_max)

    return table,mono_detection_group,pluri_detection_group,mono_detection_surface,pluri_detection_surface

mono_detection_group=pn.indicators.Number(name='Nombre de Mono détection', value=0, format='{value}',colors=[(0,'red')])
pluri_detection_group=pn.indicators.Number(name='Nombre de Pluri détections', value=0, format='{value}',colors=[(0,'green')])
mono_detection_surface=pn.indicators.Number(name='Surface (ha) Mono détection', value=0, format='{value}',colors=[(0,'red')])
pluri_detection_surface=pn.indicators.Number(name='Surface (ha) Pluri détections', value=0, format='{value}',colors=[(0,'green')])

def update_interface(event):
    table, mono_nb, pluri_nb,mono_surf, pluri_surf = maj_table(datetime_range_picker.value) 
    mono_detection_group.value = mono_nb
    pluri_detection_group.value = pluri_nb
    
    mono_detection_surface.value = mono_surf
    pluri_detection_surface.value = pluri_surf

    main.clear() 
    main.append(table)

datetime_range_picker = pn.widgets.DatetimeRangePicker(name='Select your Date Range', start=dt.datetime(2023, 1, 1), end=dt.datetime(2023, 12, 31))
datetime_range_picker.param.watch(update_interface, 'value')

### Indicateurs ########

sidebar = pn.Column(datetime_range_picker,"# Indicateurs Globaux", mono_detection_group, mono_detection_surface,pluri_detection_group,pluri_detection_surface)
main = pn.Column("Select un intervalle de date pour voir les données") 

template =pn.template.FastListTemplate(
    site="Panel", title="Dashboard Contrôle des surfaces brûlées en sortie en chaîne",logo="../quarto_project/img/OEIL_logo.png",sidebar=[sidebar],main=[main])

template.servable()

FastListTemplate
    [js_area] HTML(None, height=0, margin=0, sizing_mode='fixed', width=0)
    [actions] TemplateActions()
    [browser_info] BrowserInfo()
    [busy_indicator] LoadingSpinner(height=20, width=20)
    [main-2065754443984] Column
        [0] Markdown(str)
    [nav-2065754389008] Column
        [0] DatetimeRangePicker(as_numpy_datetime64=False, end=datetime.datetime(2023, ..., name='Select your Date Range', start=datetime.datetime(2023, ...)
        [1] Markdown(str)
        [2] Number(colors=[(0, 'red')], name='Nombre de Mono d..., value=0)
        [3] Number(colors=[(0, 'red')], name='Surface (ha) M..., value=0)
        [4] Number(colors=[(0, 'green')], name='Nombre de Pluri d..., value=0)
        [5] Number(colors=[(0, 'green')], name='Surface (ha) P..., value=0)